In [3]:
# neatness of code:
%load_ext pycodestyle_magic
%pycodestyle_on

In [1]:
# Packages:
import sqlite3
import pandas as pd
import numpy as np
# First que list all objects in the database:
conn = sqlite3.connect('../db.sqlite3')
pd.read_sql(""" SELECT * FROM sqlite_master;""", conn)
# load tables:
long = pd.read_sql("""SELECT * FROM CountryIndicators;""", conn)
# Not countries
exclude_list = ['Arab World', 'Caribbean small states', 'Central Europe and the Baltics',
 'East Asia & Pacific \(all income levels',
 'East Asia & Pacific \(developing only', 'Euro area',
 'Europe & Central Asia \(all income levels',
 'Europe & Central Asia \(developing only', 'European Union',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries \(HIPC', 'High income',
 'High income: nonOECD', 'High income: OECD',
 'Latin America & Caribbean \(all income levels',
 'Latin America & Caribbean \(developing only',
 'Least developed countries: UN classification', 'Low & middle income',
 'Low income', 'Lower middle income',
 'Middle East & North Africa \(all income levels',
 'Middle East & North Africa \(developing only', 'Middle income',
 'North America' 'OECD members' ,'Other small states',
 'Pacific island small states', 'Small states', 'South Asia',
 'Sub-Saharan Africa \(all income levels',
 'Sub-Saharan Africa \(developing only' ,'Upper middle income' ,'World', 'North America', 'OECD members']
# LongName to CountryName
country_names = pd.read_sql("""SELECT * FROM Countries;""", conn)
not_country = country_names.loc[country_names["LongName"].isin(exclude_list)]["CountryCode"]
long = long.loc[~long["CountryCode"].isin(not_country)]

In [ ]:
READ DATABASE

In [2]:
df = long.pivot_table(index=['CountryCode','Year'], columns='IndicatorCode',values='Value',aggfunc=np.sum)
df['Country'] = df.index.get_level_values(0)
df.columns = df.columns.str.replace(".", "_")
df

IndicatorCode     AG_AGR_TRAC_NO  AG_CON_FERT_PT_ZS  AG_CON_FERT_ZS  \
CountryCode Year                                                      
ABW         1960             NaN                NaN             NaN   
            1961             NaN                NaN             NaN   
            1962             NaN                NaN             NaN   
            1963             NaN                NaN             NaN   
            1964             NaN                NaN             NaN   
...                          ...                ...             ...   
ZWE         2006             NaN         210.442742       32.356341   
            2007             NaN         171.109177       27.035250   
            2008             NaN         147.903481       21.994118   
            2009             NaN         186.693038       28.778049   
            2010             NaN         215.715190       34.083000   

IndicatorCode     AG_LND_AGRI_K2  AG_LND_AGRI_ZS  AG_LND_ARBL_HA  \
CountryCode Year                                                   
ABW         1960             NaN             NaN             NaN   
            1961            20.0       11.111111          2000.0   
            1962            20.0       11.111111          2000.0   
            1963            20.0       11.111111          2000.0   
            1964            20.0       11.111111          2000.0   
...                          ...             ...             ...   
ZWE         2006        163000.0       42.135195       4100000.0   
            2007        162000.0       41.876696       4000000.0   
            2008        164500.0       42.522942       4250000.0   
            2009        163000.0       42.135195       4100000.0   
            2010        162000.0       41.876696       4000000.0   

IndicatorCode     AG_LND_ARBL_HA_PC  AG_LND_ARBL_ZS  AG_LND_CREL_HA  \
CountryCode Year                                                      
ABW         1960                NaN             NaN             NaN   
            1961           0.036078       11.111111             NaN   
            1962           0.035571       11.111111             NaN   
            1963           0.035275       11.111111             NaN   
            1964           0.035070       11.111111             NaN   
...                             ...             ...             ...   
ZWE         2006           0.312311       10.598423       2289312.0   
            2007           0.300802       10.339925       1949453.0   
            2008           0.314921       10.986170       2235440.0   
            2009           0.298812       10.598423       2080405.0   
            2010           0.286248       10.339925       1891766.0   

IndicatorCode     AG_LND_CROP_ZS  ...  per_lm_alllm_adq_pop_tot  \
CountryCode Year                  ...                             
ABW         1960             NaN  ...                       NaN   
            1961             NaN  ...                       NaN   
            1962             NaN  ...                       NaN   
            1963             NaN  ...                       NaN   
            1964             NaN  ...                       NaN   
...                          ...  ...                       ...   
ZWE         2006        0.258498  ...                       NaN   
            2007        0.258498  ...                       NaN   
            2008        0.258498  ...                       NaN   
            2009        0.258498  ...                       NaN   
            2010        0.258498  ...                       NaN   

IndicatorCode     per_lm_alllm_ben_q1_tot  per_lm_alllm_cov_pop_tot  \
CountryCode Year                                                      
ABW         1960                      NaN                       NaN   
            1961                      NaN                       NaN   
            1962                      NaN                       NaN   
            1963                      Na

LINEAR MIXED MODEL

eliminamos años donde la respuesta es NA!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [14]:
import statsmodels.formula.api as smf

# Creation of covariables for the linear model.
df['lag1'] = df['NY_GDP_MKTP_KD_ZG'].shift(1)
df['Time'] = df.index.get_level_values(1)
df = df.dropna(subset=["NY_GDP_MKTP_KD_ZG","lag1"])
# Countries strings to numeric values:
groups = df[["Country"]].replace(pd.unique(df.Country), 
            list(range(0,len(pd.unique(df.Country)))))
groups = pd.to_numeric(groups.Country)
# mixed linear model with group as random effect.
df_sub = df[["NY_GDP_MKTP_KD_ZG","Country","lag1"]]
md = smf.mixedlm("NY_GDP_MKTP_KD_ZG ~ lag1", df_sub, groups=groups)#re_formula="~Time",groups=groups,  #df["Country"].to_numpy()
mod = md.fit()
print(mod.summary())
df['residuals'] = mod.resid
df['Country'] = groups
mod.summary()

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: NY_GDP_MKTP_KD_ZG
No. Observations: 7967    Method:             REML             
No. Groups:       216     Scale:              39.7507          
Min. group size:  1       Log-Likelihood:     -26053.5410      
Max. group size:  49      Converged:          Yes              
Mean group size:  36.9                                         
-----------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|   [0.025   0.975]
-----------------------------------------------------------------
Intercept     2.908      0.113   25.829   0.000    2.687    3.128
lag1          0.277      0.011   24.737   0.000    0.255    0.299
Group Var     1.128      0.038                                   



<ipython-input-14-88c8ff961653>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['residuals'] = mod.resid
<ipython-input-14-88c8ff961653>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = groups


<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
===============================================================
Model:            MixedLM Dependent Variable: NY_GDP_MKTP_KD_ZG
No. Observations: 7967    Method:             REML             
No. Groups:       216     Scale:              39.7507          
Min. group size:  1       Log-Likelihood:     -26053.5410      
Max. group size:  49      Converged:          Yes              
Mean group size:  36.9                                         
-----------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|   [0.025   0.975]
-----------------------------------------------------------------
Intercept     2.908      0.113   25.829   0.000    2.687    3.128
lag1          0.277      0.011   24.737   0.000    0.255    0.299
Group Var     1.128      0.038                                   
===============================================================

"""

4:11: E225 missing whitespace around operator
6:43: E231 missing whitespace after ','
8:56: W291 trailing whitespace
9:13: E128 continuation line under-indented for visual indent
9:25: E231 missing whitespace after ','
12:33: E231 missing whitespace after ','
12:43: E231 missing whitespace after ','
13:68: E261 at least two spaces before inline comment
13:68: E262 inline comment should start with '# '
13:80: E501 line too long (128 > 79 characters)


FEATURE SELECTION
But in short, the mutual information between two variables, measures how much a given feature can explain another (target), or more technically, how much information about the target will variable will be obtained by having observed a feature.

The mutual information between two random variables is a non-negative value that measures the dependencies between variables. The mutual information between independent random variables is equal to 0, and the larger the mutual information, the stronger the dependency.
This function relies on the nonparametric method of entropy estimation based on k-nearest neighbor distance


In [15]:
from sklearn.feature_selection import f_regression, mutual_info_regression

# Filter/impute vars with NA
df_fewNA = df[df.columns[(df.isnull().sum(axis=0)/df.shape[0]<=0.3)]] # 0.3:210 vars
country2 = df_fewNA['Country']
df_fewNA = df_fewNA.groupby(country2).transform(lambda x: x.fillna(x.ffill().bfill()))
df_fewNA = df_fewNA.fillna(df.mean())
df_fewNA["Country"] = country2
df["residuals"]

CountryCode  Year
ABW          1996     5.231704
             1997     2.266547
             1998    -2.928074
             1999     1.956403
             2000    -3.851754
                       ...    
ZWE          2006    -4.075514
             2007    -4.889673
             2008   -18.852224
             2009     8.678401
             2010     7.526461
Name: residuals, Length: 7967, dtype: float64

4:62: E225 missing whitespace around operator
4:70: E261 at least two spaces before inline comment
4:80: E501 line too long (84 > 79 characters)
6:80: E501 line too long (86 > 79 characters)


In [21]:
# Feature selection
covs = df_fewNA.drop(["NY_GDP_MKTP_KD_ZG","residuals"], 1)
Y = df_fewNA[['residuals']]
info = mutual_info_regression(covs, np.ravel(Y))
df_varimp =pd.DataFrame(data={'name': covs.columns, 'varimp': info})
# Keep top50
selected_variables = df_varimp.sort_values(by="varimp",ascending=False)[0:49]
selected_variables['name'] = selected_variables['name'].str.replace('_','.')

2:42: E231 missing whitespace after ','
5:12: E225 missing whitespace around operator
7:55: E231 missing whitespace after ','
8:72: E231 missing whitespace after ','


In [ ]:
FEATURE SELECTION: NAMES OF FINAL VARIABLES

In [18]:
# Definition of variables: sql: Indicators
conn = sqlite3.connect('../db.sqlite3')
list_of_entries_to_retrive = (selected_variables['name']).tolist()
queryString = 'SELECT IndicatorCode,LongDefinition FROM Indicators WHERE IndicatorCode IN (\'{}\');'.format('\',\''.join([_ for _ in list_of_entries_to_retrive]))
vars_definition = pd.read_sql(queryString, con=conn)
vars_definition = vars_definition.append(pd.DataFrame(data={'IndicatorCode': ["lag1","Country","Time"],
                                    'LongDefinition': ["lag1","Country","Time"]}), ignore_index=True)
vars_definition = vars_definition.sort_values(by="IndicatorCode")
#vars_definition.columns = ['IndicatorCode','name']
# bind cols
selected_variables = selected_variables.sort_values(by="name")
if selected_variables['name'].tolist() == vars_definition['IndicatorCode'].tolist():
    selected_variables["LongDefinition"] = vars_definition['LongDefinition'].tolist()
#df_c = pd.concat([selected_variables.reset_index(drop=True), vars_definition['LongDefinition'].reset_index(drop=True) ], axis=1)

# NY_GDP_MKTP_KD_ZG
selected_variables



,name,varimp,LongDefinition
0,AG.LND.AGRI.K2,0.047800,Agricultural land refers to the share of land ...
1,AG.LND.AGRI.ZS,0.042834,Agricultural land refers to the share of land ...
3,AG.LND.ARBL.HA.PC,0.044932,Arable land (hectares per person) includes lan...
13,AG.YLD.CREL.KG,0.041383,"Cereal yield, measured as kilograms per hectar..."
207,Country,0.048448,Country
16,DC.DAC.DEUL.CD,0.053135,Net bilateral aid flows from DAC donors are th...
17,DC.DAC.JPNL.CD,0.056741,Net bilateral aid flows from DAC donors are th...
18,DC.DAC.TOTL.CD,0.062019,Net bilateral aid flows from DAC donors are th...
19,DT.ODA.ALLD.CD,0.055408,Net official development assistance (ODA) cons...
20,DT.ODA.ALLD.KD,0.049934,Net official development assistance (ODA) cons...


4:80: E501 line too long (162 > 79 characters)
6:80: E501 line too long (103 > 79 characters)
6:85: E231 missing whitespace after ','
6:95: E231 missing whitespace after ','
7:37: E128 continuation line under-indented for visual indent
7:62: E231 missing whitespace after ','
7:72: E231 missing whitespace after ','
7:80: E501 line too long (101 > 79 characters)
9:1: E265 block comment should start with '# '
12:80: E501 line too long (84 > 79 characters)
13:80: E501 line too long (85 > 79 characters)
14:1: E265 block comment should start with '# '
14:80: E501 line too long (129 > 79 characters)
19:1: W391 blank line at end of file


READ BDD FOR FINAL MODEL: SQL:CountryIndicators read only important variables

In [22]:
selected_variables

,name,varimp
125,NY.GDP.PCAP.KD.ZG,0.959658
89,NV.IND.TOTL.KD.ZG,0.258287
93,NV.SRV.TETC.KD.ZG,0.220813
83,NV.AGR.TOTL.KD.ZG,0.088687
205,lag1,0.078651
206,Time,0.074365
18,DC.DAC.TOTL.CD,0.062316
17,DC.DAC.JPNL.CD,0.056863
19,DT.ODA.ALLD.CD,0.056059
186,TM.VAL.MRCH.CD.WT,0.053002


In [6]:
#### 1) We find Y: EstimatedGDPGrowth for the next year:
#summ_vars2 = summ_vars[summ_vars['IndicatorName'].str.contains("GDP")]
#summ_vars2[~summ_vars2.IndicatorName.str.contains("% of GDP")]
    # Response variable: GDP growth (annual %): NY.GDP.MKTP.KD.ZG
    # GPD covariate: GDP (constant 2005 US$), NY.GDP.MKTP.KD [look description of response if any doubt].
#long.CountryCode.unique()[1:10]

ERROR! Session/line number was not unique in database. History logging moved to new session 41
7.18.1


1:1: E266 too many leading '#' for block comment
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
4:5: E116 unexpected indentation (comment)
5:5: E116 unexpected indentation (comment)
5:80: E501 line too long (105 > 79 characters)
6:1: E265 block comment should start with '# '
7:11: E225 missing whitespace around operator
